# Calculate ILAMB Data
Creates annual, regridded data as well as month of max and climatology, from raw ILAMB data. Uses a config file (e.g. ILAMB_obs.cfg) to get information about where ILAMB data is located and what the files, models, and variable names are.

In [ ]:
import os
import warnings
import xarray as xr

import fates_calibration_library.utils as utils
import fates_calibration_library.ilamb_functions as ilamb
import fates_calibration_library.analysis_functions as analysis

# ignore warnings about latitude being outside of -90, 90
warnings.simplefilter("ignore", UserWarning)

In [ ]:
# config file for ILAMB data
ilamb_cfg = '/glade/work/afoster/FATES_calibration/fates_calibration_library/configs/ILAMB_obs.yaml'

# land area file
land_area_ds = os.path.join("/glade/derecho/scratch/afoster/archive",
                            "ctsm60SP_bigleaf_fullgrid/lnd/hist",
                            "ctsm60SP_bigleaf_fullgrid.clm2.h0.0001-02-01-00000.nc")

In [ ]:
# config dictionary for running functions
config_dict = {
    'top_dir': '/glade/campaign/cesm/community/lmwg/diag/ILAMB/DATA',  # location of raw ILAMB data
    'regrid_dir': '/glade/work/afoster/FATES_calibration/observations/ILAMB_obs/regridded',  # where to put regridded ILAMB data
    'regrid_tag': '2deg',
    'out_dir': '/glade/work/afoster/FATES_calibration/observations/ILAMB_obs',  # where to put compiled ILAMB data
    'user': 'afoster@ucar.edu',  # user
    'clobber': False,  # overwrite files?
}

In [ ]:
# read in the ILAMB data dictionary
ilamb_dict = utils.get_config_file(ilamb_cfg)

In [ ]:
# create a target grid to regrid to
target_grid = analysis.create_target_grid(land_area_ds, 'FSR')

In [ ]:
# loop through dictionary and regrid the ILAMB observations
# script will skip files that already exist unless clobber: True
ilamb.regrid_all_ilamb_data(config_dict, ilamb_dict, target_grid)

In [ ]:
dataset = 'FLUXCOM_LE'
attributes = ilamb_dict[dataset]

In [ ]:
#ilamb.get_all_ilamb_data(config_dict, ilamb_dict)

In [ ]:
# # compile data into one dataset
# compiled_ds = ilamb.compile_ilamb_datasets(config_dict['out_dir'], ilamb_dict,
#                                            target_grid.area)
# compiled_ds.to_netcdf(os.path.join('/glade/work/afoster/FATES_calibration',
#                                    'observations/all_ILAMB_obs.nc'), mode="w")

In [ ]:
region = xr.open_dataset(
    os.path.join(config_dict["top_dir"], "regions/GlobalLandNoAnt.nc")
)
# swap out 0.0 (land) for 1.0
region["ids"] = xr.where(region.ids == 0.0, 1.0, 0.0)

In [ ]:
# read in regridded dataset
file_name = os.path.join(
    config_dict["regrid_dir"],
    f"{attributes['model']}_{attributes['out_var'].upper()}_{config_dict['regrid_tag']}.nc",
)

In [ ]:
regridded_dat = xr.open_dataset(file_name)

In [ ]:
metadata = {
    "units": attributes["units"],
    "longname": attributes["longname"],
    "original_file": file_name,
    "user": config_dict["user"],
}

In [ ]:
annual_ds = ilamb.get_annual_ds(
    regridded_dat,
    attributes["in_var"],
    attributes["out_var"],
    ilamb.evaluate_conversion_factor(attributes["time_conversion_factor"]),
    metadata,
)

In [ ]:
ilamb_dat, original_file = ilamb.read_ilamb_data(config_dict["top_dir"], attributes)
annual_original = ilamb.get_annual_ds(
    ilamb_dat,
    attributes["in_var"],
    attributes["out_var"],
    ilamb.evaluate_conversion_factor(attributes["time_conversion_factor"]),
    metadata,
)

In [ ]:
# calculate cell areas and land area
cell_area = ilamb.get_ilamb_land_area(ilamb_dat)
region_regridder = xe.Regridder(region["ids"], ilamb_dat, "bilinear")
landfrac = region_regridder(region["ids"])
land_area = landfrac * cell_area

In [ ]:
global_mean = analysis.area_mean(
    annual_original[attributes["out_var"]],
    ilamb.evaluate_conversion_factor(attributes["area_conversion_factor"]),
    land_area,
).to_dataset(name=f"{attributes['out_var']}_global")

In [ ]:
data_array = ilamb_dat[attributes["in_var"]]
area_cf = ilamb.evaluate_conversion_factor(attributes["area_conversion_factor"])
conversion_factor = ilamb.evaluate_conversion_factor(attributes["time_conversion_factor"])

In [ ]:
if conversion_factor is None:
    conversion_factor = 1.0

data_array = data_array * conversion_factor

days_in_month = data_array["time"].dt.days_in_month
weighted = data_array * days_in_month
valid_days = days_in_month.where(data_array.notnull())

In [ ]:
monthly_sum = weighted.groupby("time.month").sum(dim="time", skipna=True)
total_days = valid_days.groupby("time.month").sum(dim="time", skipna=True)